In [ ]:
from google.colab import files
import zipfile
from google.colab import drive

# drive.mount('/content/drive/')
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
import os
import numpy as np
import cv2
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
import matplotlib.pyplot as plt
Learning_Rate=1e-5
width = 900
height = 900 # image width and height
batchSize=3

In [ ]:
TrainFolder = "/content/drive/MyDrive/Semester 7/Robotics and Computer Vision/Robotics Final Project/LabPicsV1/Simple/Train//"
ListImages=os.listdir(os.path.join(TrainFolder, "Image"))

In [ ]:
transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)),tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
transformAnn=tf.Compose([tf.ToPILImage(),tf.Resize((height,width),tf.InterpolationMode.NEAREST),tf.ToTensor()])

In [ ]:
def ReadRandomImage(): # First lets load random image and  the corresponding annotation
    idx=np.random.randint(0,len(ListImages)) # Select random image
    Img=cv2.imread(os.path.join(TrainFolder, "Image", ListImages[idx]))[:,:,0:3]
    Filled =  cv2.imread(os.path.join(TrainFolder, "Semantic/16_Filled", ListImages[idx].replace("jpg","png")),0)
    Vessel =  cv2.imread(os.path.join(TrainFolder, "Semantic/1_Vessel", ListImages[idx].replace("jpg","png")),0)
    AnnMap = np.zeros(Img.shape[0:2],np.float32)
    if Vessel is not None:  AnnMap[ Vessel == 1 ] = 1
    if Filled is not None:  AnnMap[ Filled  == 1 ] = 2
    Img=transformImg(Img)
    AnnMap=transformAnn(AnnMap)
    return Img,AnnMap

In [ ]:
def LoadBatch(): # Load batch of images
    images = torch.zeros([batchSize,3,height,width])
    ann = torch.zeros([batchSize, height, width])
    for i in range(batchSize):
        images[i],ann[i]=ReadRandomImage()
    return images, ann

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
Net = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True) # Load net
Net.classifier[4] = torch.nn.Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1)) # Change final layer to 3 classes
Net=Net.to(device)
optimizer=torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate) # Create adam optimizer

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth


  0%|          | 0.00/161M [00:00<?, ?B/s]

In [ ]:
for itr in range(1000): # Training loop
   images,ann=LoadBatch() # Load taining batch
   images=torch.autograd.Variable(images,requires_grad=False).to(device) # Load image
   ann = torch.autograd.Variable(ann, requires_grad=False).to(device) # Load annotation
   Pred=Net(images)['out'] # make prediction
   Net.zero_grad()
   criterion = torch.nn.CrossEntropyLoss() # Set loss function
   Loss=criterion(Pred,ann.long()) # Calculate cross entropy loss
   Loss.backward() # Backpropogate loss
   optimizer.step() # Apply gradient descent change to weight
   seg = torch.argmax(Pred[0], 0).cpu().detach().numpy()  # Get  prediction classes
   print(itr,") Loss=",Loss.data.cpu().numpy())
   if itr % 100 == 0: #Save model weight once every 60k steps permenant file
        print("Saving Model" +str(itr) + ".torch")
        torch.save(Net.state_dict(),   str(itr) + ".torch")

0 ) Loss= 1.0496049
Saving Model0.torch
1 ) Loss= 1.0343045
2 ) Loss= 1.0461655
3 ) Loss= 1.0248842
4 ) Loss= 1.0563486
5 ) Loss= 1.030735
6 ) Loss= 1.0885108
7 ) Loss= 1.0031171
8 ) Loss= 1.0117192
9 ) Loss= 0.9572755
10 ) Loss= 0.99551904
11 ) Loss= 0.98815185
12 ) Loss= 1.003546
13 ) Loss= 0.9942357
14 ) Loss= 1.0079284
15 ) Loss= 0.9681646
16 ) Loss= 0.95045596
17 ) Loss= 0.927225
18 ) Loss= 1.0096952
19 ) Loss= 0.960226
20 ) Loss= 0.9756343
21 ) Loss= 0.9061643
22 ) Loss= 0.92311245
23 ) Loss= 0.9632667
24 ) Loss= 1.0081809
25 ) Loss= 0.9508215
26 ) Loss= 0.9063601
27 ) Loss= 0.90920043
28 ) Loss= 0.9067501
29 ) Loss= 0.9083868
30 ) Loss= 0.90647614
31 ) Loss= 0.91197914
32 ) Loss= 0.924464
33 ) Loss= 0.8632034
34 ) Loss= 0.8758635
35 ) Loss= 0.9060454
36 ) Loss= 0.8390711
37 ) Loss= 0.8544678
38 ) Loss= 0.9567294
39 ) Loss= 0.95170814
40 ) Loss= 0.889328
41 ) Loss= 0.87063485
42 ) Loss= 0.8893701
43 ) Loss= 0.8305523
44 ) Loss= 0.7949268
45 ) Loss= 0.7829823
46 ) Loss= 0.92752385